In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')                                                                                                                                                                                        

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!apt-get install openjdk-8-jdk -qq > /dev/null
!pip install h2o -qq

In [ ]:
!npm install -g localtunnel -qq > /dev/null
get_ipython().system_raw('lt --port 54321 >> url.txt 2>&1 &')
!cat url.txt

In [1]:
import os

print(os.getcwd())
os.chdir('gdrive/My Drive/financial-predictions')
print(os.getcwd())

/home/edu/Documents/1-projects-ironhack/1-final-project/1-predicting-markets-DL/workflow


FileNotFoundError: [Errno 2] No such file or directory: 'gdrive/My Drive/financial-predictions'

In [2]:
import pandas as pd
from timeSeriesToSupervised import series_to_supervised
from transformData import minMaxScaleOnTrain
import numpy as np
import splitSequence as sq
import h2o
from h2o.automl import H2OAutoML
import ta

Using TensorFlow backend.


In [3]:
symbol = 'aep'
stock = pd.read_csv(f'../inputData/stocksLong/{symbol}.us.txt', parse_dates=True).drop(columns='OpenInt')
stock.set_index('Date', inplace=True)
display(stock.head())
stock  = ta.add_trend_ta(stock, high='High', low='Low', close='Close', fillna=True)
stock = ta.add_others_ta(stock, close='Close', fillna=True)
stock.columns
#print(stock.shape)

,Open,High,Low,Close,Volume
Date,,,,,
1970-01-02,1.5325,1.5783,1.5325,1.5631,13511
1970-01-05,1.5631,1.5783,1.5403,1.5783,23083
1970-01-06,1.5783,1.5934,1.5708,1.5708,18360
1970-01-07,1.5708,1.5783,1.5552,1.5783,10098
1970-01-08,1.5708,1.5708,1.5552,1.5552,21375


/home/edu/.local/lib/python3.7/site-packages/ta/trend.py:543: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
/home/edu/.local/lib/python3.7/site-packages/ta/trend.py:547: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'trend_macd',
       'trend_macd_signal', 'trend_macd_diff', 'trend_ema_fast',
       'trend_ema_slow', 'trend_adx', 'trend_adx_pos', 'trend_adx_neg',
       'trend_vortex_ind_pos', 'trend_vortex_ind_neg', 'trend_vortex_ind_diff',
       'trend_trix', 'trend_mass_index', 'trend_cci', 'trend_dpo', 'trend_kst',
       'trend_kst_sig', 'trend_kst_diff', 'trend_ichimoku_a',
       'trend_ichimoku_b', 'trend_visual_ichimoku_a',
       'trend_visual_ichimoku_b', 'trend_aroon_up', 'trend_aroon_down',
       'trend_aroon_ind', 'trend_psar', 'trend_psar_up', 'trend_psar_down',
       'trend_psar_up_indicator', 'trend_psar_down_indicator', 'others_dr',
       'others_dlr', 'others_cr'],
      dtype='object')

In [4]:
columns = ['Open', 'High', 'Low', 'Volume', 'trend_macd', 'others_dr']
df_forML = series_to_supervised(list(stock['Close']), n_in=50, n_out=1, name_var='Close')
df_forML.rename(columns={'Close(t)':'y'}, inplace=True) 
for col in columns:
    df_forML = df_forML.merge(series_to_supervised(list(stock[col]), n_in=50, n_out=0, name_var=col), left_index=True, right_index=True)
df_forML = df_forML.set_index(stock.iloc[50:].index)
df_forML = df_forML.iloc[:int(len(df_forML) * 0.8)]
df_forML.head()

,Close(t-50),Close(t-49),Close(t-48),Close(t-47),Close(t-46),Close(t-45),Close(t-44),Close(t-43),Close(t-42),Close(t-41),...,others_dr(t-10),others_dr(t-9),others_dr(t-8),others_dr(t-7),others_dr(t-6),others_dr(t-5),others_dr(t-4),others_dr(t-3),others_dr(t-2),others_dr(t-1)
Date,,,,,,,,,,,,,,,,,,,,,
1970-03-16,1.5631,1.5783,1.5708,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,...,4.449588,-0.960355,0.969667,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973
1970-03-17,1.5783,1.5708,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,...,-0.960355,0.969667,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000
1970-03-18,1.5708,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,1.5248,...,0.969667,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000,-0.505406
1970-03-19,1.5783,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,1.5248,1.4942,...,-1.434376,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000,-0.505406,0.000000
1970-03-20,1.5552,1.5708,1.5552,1.5097,1.5097,1.5019,1.4942,1.5248,1.4942,1.5097,...,-2.373368,0.000000,-0.505406,0.000000,0.000000,0.507973,0.000000,-0.505406,0.000000,-0.958076


In [5]:
# We scale the data based on the training period.
df_forML = minMaxScaleOnTrain(df_forML)

In [15]:
df_forML

,Close(t-50),Close(t-49),Close(t-48),Close(t-47),Close(t-46),Close(t-45),Close(t-44),Close(t-43),Close(t-42),Close(t-41),...,others_dr(t-10),others_dr(t-9),others_dr(t-8),others_dr(t-7),others_dr(t-6),others_dr(t-5),others_dr(t-4),others_dr(t-3),others_dr(t-2),others_dr(t-1)
Date,,,,,,,,,,,,,,,,,,,,,
1970-03-16,0.020594,0.021113,0.020857,0.021113,0.020324,0.020857,0.020324,0.018769,0.018769,0.018502,...,0.740725,0.477243,0.571241,0.454157,0.408425,0.524016,0.499401,0.524016,0.524016,0.548756
1970-03-17,0.021113,0.020857,0.021113,0.020324,0.020857,0.020324,0.018769,0.018769,0.018502,0.018239,...,0.477243,0.571241,0.454157,0.408425,0.524016,0.499401,0.524016,0.524016,0.548756,0.524016
1970-03-18,0.020857,0.021113,0.020324,0.020857,0.020324,0.018769,0.018769,0.018502,0.018239,0.019285,...,0.571241,0.454157,0.408425,0.524016,0.499401,0.524016,0.524016,0.548756,0.524016,0.499401
1970-03-19,0.021113,0.020324,0.020857,0.020324,0.018769,0.018769,0.018502,0.018239,0.019285,0.018239,...,0.454157,0.408425,0.524016,0.499401,0.524016,0.524016,0.548756,0.524016,0.499401,0.524016
1970-03-20,0.020324,0.020857,0.020324,0.018769,0.018769,0.018502,0.018239,0.019285,0.018239,0.018769,...,0.408425,0.524016,0.499401,0.524016,0.524016,0.548756,0.524016,0.499401,0.524016,0.477354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-04-21,1.105447,1.092014,1.099055,1.111018,1.110232,1.088869,1.090408,1.078957,1.068259,1.048160,...,0.518358,0.548761,0.502659,0.592242,0.600996,0.469059,0.544201,0.619575,0.489299,0.554809
2008-04-22,1.092014,1.099055,1.111018,1.110232,1.088869,1.090408,1.078957,1.068259,1.048160,1.069626,...,0.548761,0.502659,0.592242,0.600996,0.469059,0.544201,0.619575,0.489299,0.554809,0.491271
2008-04-23,1.099055,1.111018,1.110232,1.088869,1.090408,1.078957,1.068259,1.048160,1.069626,1.083913,...,0.502659,0.592242,0.600996,0.469059,0.544201,0.619575,0.489299,0.554809,0.491271,0.484140


In [6]:
h2o.init(
    nthreads=7)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu119.10.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu119.10.1, mixed mode, sharing)
  Starting server from /home/edu/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2nkeysrr
  JVM stdout: /tmp/tmp2nkeysrr/h2o_edu_started_from_python.out
  JVM stderr: /tmp/tmp2nkeysrr/h2o_edu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,1 month and 7 days
H2O cluster name:,H2O_from_python_edu_qhhn4o
H2O cluster total nodes:,1
H2O cluster free memory:,1.887 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,7
H2O cluster status:,"accepting new members, healthy"


In [7]:
dfh2o = h2o.H2OFrame(df_forML)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
train, test = dfh2o.split_frame(ratios=[0.8])

In [10]:
aml = H2OAutoML(max_models=10)
aml.train(y= 'y', training_frame=train, leaderboard_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200312_230448,6.3965e-05,0.00799781,6.3965e-05,0.00417862,0.00446342
StackedEnsemble_BestOfFamily_AutoML_20200312_230448,6.40138e-05,0.00800086,6.40138e-05,0.00417714,0.00446467
XGBoost_1_AutoML_20200312_230448,6.46272e-05,0.0080391,6.46272e-05,0.00419165,0.00448178
GBM_2_AutoML_20200312_230448,6.59733e-05,0.00812239,6.59733e-05,0.00418552,0.00455064
XGBoost_2_AutoML_20200312_230448,6.61942e-05,0.00813598,6.61942e-05,0.00424157,0.00456338
XGBoost_3_AutoML_20200312_230448,6.65727e-05,0.00815921,6.65727e-05,0.00424192,0.00454698
GBM_3_AutoML_20200312_230448,6.73418e-05,0.00820621,6.73418e-05,0.00420788,0.00457659
GBM_4_AutoML_20200312_230448,7.00897e-05,0.00837196,7.00897e-05,0.00421867,0.00466633
DRF_1_AutoML_20200312_230448,7.1045e-05,0.00842882,7.1045e-05,0.00414325,0.00470128
GBM_1_AutoML_20200312_230448,7.26104e-05,0.00852117,7.26104e-05,0.00423397,0.00477425


In [12]:
aml.event_log

timestamp,level,stage,message,name,value
23:04:48.476,Info,Workflow,Project: AutoML_20200312_230448475,,
23:04:48.476,Info,Validation,Setting stopping tolerance adaptively based on the training frame: 0.011451215811497927,,
23:04:48.476,Info,Validation,Build control seed: -1 (random),,
23:04:48.477,Info,DataImport,training frame: Frame key: automl_training_py_3_sid_b0a0 cols: 351 rows: 7626 chunks: 16 size: 17383564 checksum: -4560635162867611764,,
23:04:48.477,Info,DataImport,validation frame: NULL,,
23:04:48.477,Info,DataImport,leaderboard frame: Frame key: py_4_sid_b0a0 cols: 351 rows: 1994 chunks: 16 size: 5691280 checksum: 2613432320746894096,,
23:04:48.477,Info,DataImport,blending frame: NULL,,
23:04:48.477,Info,DataImport,response column: y,,
23:04:48.477,Info,DataImport,fold column: null,,
23:04:48.477,Info,DataImport,weights column: null,,


In [13]:
os.getcwd()
model_path = h2o.save_model(model=aml.leader, path="../modelOutputs/h2oModel")
model2 = h2o.get_model(aml.leaderboard.as_data_frame()['model_id'][2])
model3 = h2o.get_model(aml.leaderboard.as_data_frame()['model_id'][3])
model4 = h2o.get_model(aml.leaderboard.as_data_frame()['model_id'][4])

In [ ]:
model_path = h2o.save_model(model=model4, path="./h2o-model")
# model_path = h2o.save_model(model=model2, path="./h2o-model")
# model_path = h2o.save_model(model=model3, path="./h2o-model")

In [14]:
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20200312_230448

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 6.042900543372696e-06
RMSE: 0.00245823118184045
MAE: 0.0014764786960559924
RMSLE: 0.0014543263026993193
R^2: 0.9999544373366437
Mean Residual Deviance: 6.042900543372696e-06
Null degrees of freedom: 7625
Residual degrees of freedom: 7618
Null deviance: 1011.4237436786708
Residual deviance: 0.04608315954376018
AIC: -69979.14271442105

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 6.660291238319852e-05
RMSE: 0.008161060738849975
MAE: 0.004196302150657545
RMSLE: 0.004642578596032392
R^2: 0.9994978229322687
Mean Residual Deviance: 6.660291238319852e-05
Null degrees of freedom: 7625
Residual degrees of freedom: 7616
Null deviance: 1011.64007091688
Residual deviance: 0.5079138098342719
AIC: -51673.77842452566


In [19]:
predictions = aml.leader.predict(test)
test['y'].cbind(predictions)

stackedensemble prediction progress: |████████████████████████████████████| 100%


y,predict
0.0198145,0.0210824
0.0221558,0.0220753
0.0234615,0.0219374
0.0229317,0.023565
0.0229317,0.0233798
0.0205938,0.0209815
0.0159076,0.0176253
0.0145951,0.016179
0.0130262,0.0143797
0.0125135,0.0128965


In [35]:
predsTrainTest = aml.leader.predict(dfh2o)
predsTrainTest = dfh2o['y'].cbind(predsTrainTest)
len(predsTrainTest)

stackedensemble prediction progress: |████████████████████████████████████| 100%


9620

In [25]:
frameForMerge = df_forML.reset_index()

In [28]:
dates = h2o.H2OFrame(frameForMerge[['Date']])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [36]:
results = dates.cbind(predsTrainTest)

Date,y,predict
1970-03-16 00:00:00,0.0205938,0.0210896
1970-03-17 00:00:00,0.0203238,0.0210645
1970-03-18 00:00:00,0.0203238,0.0209201
1970-03-19 00:00:00,0.0198145,0.0210824
1970-03-20 00:00:00,0.0205938,0.0203792
1970-03-23 00:00:00,0.0203238,0.0211514
1970-03-24 00:00:00,0.0205938,0.0210556
1970-03-25 00:00:00,0.0229317,0.0214319
1970-03-26 00:00:00,0.0218926,0.0234676
1970-03-30 00:00:00,0.0221558,0.0220616
